In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import keras

import random as rn

np.random.seed(42)
rn.seed(42)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)


from keras import backend as K

tf.set_random_seed(42)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

generate_data = False


x = np.load("data/x.npy")
y = np.load("data/y.npy")

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization

best_aucroc = 0

In [3]:
activations = ['relu']
dropouts = [0.8]
num_total_layers = [2]
losses = ['binary_crossentropy']
val_sizes = [0.33]
last_layer_sizes = [100]
last_layer_drop_sizes = [0.15]


# activations = ['tanh']
# dropouts = [0.93, 0.3, 0.99]
# num_total_layers = [4]
# losses = ['categorical_hinge']
# val_sizes = [0.4]
# last_layer_sizes = [20]
# last_layer_drop_sizes = [0.15]

In [4]:
def get_correct_split(test_size_input):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_input, random_state=42)
    return x_train, x_test, y_train, y_test

def generate_csv(model, x_data, name):
    ids = [i for i, _ in enumerate(x_data)]
    ids = np.array(ids)
    
    predictions = model.predict(x_data).flatten()
    
    df = pd.DataFrame()
    df['id'] = ids
    df['target'] = predictions
    df.to_csv(name + ".csv", index=False)

In [5]:
def create_model(activation, dropout, num_total_layer, last_layer_size, last_layer_drop_size):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(len(x[0]),)))
    model.add(Dense(300))
    model.add(Activation(activation))
    model.add(Dropout(dropout))
    
    num_layers_still_needed = num_total_layer - 2 # We manually create first and last
        
    step_size = int((300 - last_layer_size) / (num_layers_still_needed + 1))
        
    count = 300

    
    for i in range(num_layers_still_needed):
        
        count -= step_size

        model.add(Dense(count))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout))

    model.add(Dense(last_layer_size))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dropout(last_layer_drop_size))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

In [6]:
# model.summary()

In [7]:
from keras.callbacks import Callback

class Stat_Collector (Callback):
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.aucrocs_train = []

    def on_epoch_end(self, batch, logs=None):
        # Make aucroc prediction for training data
        y_pred_train = self.model.predict(x_train)
        y_pred_train = y_pred_train.flatten()
        self.aucrocs_train.append(roc_auc_score(y_train, y_pred_train))

In [8]:
def train_model(model, activation, dropout, num_total_layer, loss, val_size, last_layer_size, last_layer_drop_size, x_train, y_train):
#     stat_collect = Stat_Collector(x_train, y_train)

    model.compile(loss=loss,optimizer='adam', metrics=['accuracy'])

    fit = model.fit(x_train, y_train, batch_size=32, nb_epoch=23, verbose=1)
    
    x_actual_test = np.load("data/x_test.npy")
    
    generate_csv(model, x_actual_test, "submissions/nuro_net_test")

#     import matplotlib.pyplot as plt

#     num_epoch = len(23)

#     # Plot training & validation accuracy values
#     plt.plot([x + 1 for x in range(num_epoch)], stat_collect.aucrocs_train, label="Train AUCROC")
#     plt.plot([x + 1 for x in range(num_epoch)], stat_collect.aucrocs_val, label="Test AUCROC")

#     plt.xlabel('Test AUCROC maximized at epoch = {} with AUCROC = {}'.format(np.argmax(stat_collect.aucrocs_val) + 1, stat_collect.aucrocs_val[np.argmax(stat_collect.aucrocs_val)]))
#     plt.ylabel('AUCROC')
#     plt.title('Epochs vs. AUCROC')
#     plt.legend(['Train', 'Test'], loc='upper left')
#     plt.savefig("plots/nani")
#     plt.show()
#     global best_aucroc
#     if best_aucroc < stat_collect.aucrocs_val[np.argmax(stat_collect.aucrocs_val)]:
#         best_aucroc = stat_collect.aucrocs_val[np.argmax(stat_collect.aucrocs_val)]
#         with open("plots/best_plot.txt", "a+") as f:
#             f.write("plots/activation:{}_dropout:{}_numlayer:{}_loss:{}_valsize:{}_lastlayersize:{}_lastlayerdrop:{} \n Performance: {}\n".format(activation, dropout, num_total_layer, loss, val_size, last_layer_size, last_layer_drop_size, stat_collect.aucrocs_val[np.argmax(stat_collect.aucrocs_val)]))


In [9]:
for activation in activations:
    for dropout in dropouts:
        for num_total_layer in num_total_layers:
            for loss in losses:
                for val_size in val_sizes:
                    for last_layer_size in last_layer_sizes:
                        for last_layer_drop_size in last_layer_drop_sizes:
#                             x_train, x_test, y_train, y_test = get_correct_split(val_size)
#                             print(activation, dropout, num_total_layer, loss, val_size, last_layer_size, last_layer_drop_size)
                            model = create_model(activation, dropout, num_total_layer, last_layer_size, last_layer_drop_size)
                            train_model(model, activation, dropout, num_total_layer, loss, val_size, last_layer_size, last_layer_drop_size, x, y)
#                             model.summary()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/23
64667/64667 [==============================] - 9s 142us/step - loss: 0.5213 - acc: 0.7503
Epoch 2/23
64667/64667 [==============================] - 7s 110us/step - loss: 0.4932 - acc: 0.7636
Epoch 3/23
64667/64667 [==============================] - 7s 103us/step - loss: 0.4889 - acc: 0.7666
Epoch 4/23
64667/64667 [==============================] - 8s 117us/step - loss: 0.4859 - acc: 0.7684
Epoch 5/23
64667/64667 [==============================] - 8s 124us/step - loss: 0.4832 - acc: 0.7695
Epoch 6/23
64667/64667 [==============================] - 8s 121us/step - loss: 0.4832 - acc: 0.7704
Epoch 7/23
64667/64667 [==============================] - 6s 94us/step - loss: 0.4808 - acc: 0.7716
Epoch 8/23
64667/64667 [==============================] - 6s 95us/step - loss: 0.4797 - acc: 0.7726
Epoch 9/23
64667/64667 [==============================] - 6s 98us/step - loss: 0.4776 - acc: 0.7724
Epoch 10/23
64667/64667 [==============================] - 7s 102us/step - loss: 0.4770 - acc:

In [10]:
# def generate_csv(model, x_data, name):
#     ids = [i for i, _ in enumerate(x_data)]
#     ids = np.array(ids)
    
#     predictions = model.predict(x_data).flatten()
    
#     df = pd.DataFrame()
#     df['id'] = ids
#     df['target'] = predictions
#     df.to_csv(name + ".csv", index=False)

# #     print df.head()

# x_actual_test = np.load("data/x_test.npy")

# generate_csv(model, x_actual_test, "submissions/third_test")